In [1]:
import findspark

findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, lag, avg, round, count
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import folium



In [2]:
import os

def getFilePath(folder_path):
    file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    return file_paths

In [3]:
file_paths=getFilePath('./aisdk-2006-03')
file_path='./aisdk_20060302.csv'

In [4]:
spark = SparkSession.builder \
    .appName("aisdk") \
    .getOrCreate()

In [5]:
df = spark.read.csv(
    file_paths,
    header=True,        
    inferSchema=True,  
    nullValue="",       
    mode="DROPMALFORMED"  
)

In [6]:
# Navigational status 
unique_values = df.select("Navigational status").distinct()

unique_values_list = [row[0] for row in unique_values.collect()]

print(unique_values_list)


['Moored', 'Restricted maneuverability', 'Reserved for future amendment [HSC]', 'Unknown value', 'Constrained by her draught', 'Reserved for future use [11]', 'Reserved for future use [12]', 'Not under command', 'At anchor', 'Engaged in fishing', 'Under way using engine', 'Under way sailing', 'Aground', 'Reserved for future amendment [WIG]', 'AIS-SART', 'Reserved for future use [13]']


In [7]:
df = df.withColumn("timestamp", unix_timestamp("# Timestamp", "dd/MM/yyyy HH:mm:ss").cast("timestamp"))



windowSpec = Window.partitionBy("MMSI").orderBy("timestamp")



df_with_lag = df.withColumn("prev_status", lag("Navigational status").over(windowSpec)) \
                .withColumn("prev_timestamp", lag("timestamp").over(windowSpec))

#  Moored to Under way using engine
operation_time = df_with_lag.filter(
    (col("Navigational status") == "Under way using engine") & 
    (col("prev_status") == "Moored")
).withColumn(
    "operation_duration",
    (col("timestamp").cast("long") - col("prev_timestamp").cast("long")) / 3600  # 小時
)

operation_time.select("MMSI", "operation_duration").show()


+-------+--------------------+
|   MMSI|  operation_duration|
+-------+--------------------+
|1193046|0.007777777777777...|
|1193046| 0.09666666666666666|
|1193046| 0.09694444444444444|
|1193046|0.050277777777777775|
|1193046| 0.05277777777777778|
|1193046|0.002222222222222...|
|1193046|0.015833333333333335|
|1193046|0.015833333333333335|
|1193046|0.005277777777777778|
|1193046|0.005555555555555556|
|1193046|0.008333333333333333|
|1193046|0.010833333333333334|
|1193046|0.013888888888888888|
|1193046| 0.01611111111111111|
|1193046|0.035555555555555556|
|1193046| 0.03777777777777778|
|1193046|0.024722222222222222|
|1193046| 0.03027777777777778|
|1193046| 0.04638888888888889|
|1193046| 0.03277777777777778|
+-------+--------------------+
only showing top 20 rows



In [8]:
# At anchor to Moored
waiting_time = df_with_lag.filter(
    (col("Navigational status") == "Moored") & 
    (col("prev_status") == "At anchor")
).withColumn(
    "waiting_duration",
    (col("timestamp").cast("long") - col("prev_timestamp").cast("long")) / 3600  # 小時
)


waiting_time.select("MMSI", "waiting_duration").show()


+---------+--------------------+
|     MMSI|    waiting_duration|
+---------+--------------------+
|211409660| 0.03861111111111111|
|212064000| 0.10833333333333334|
|247309000| 0.10277777777777777|
|265146000|  0.0661111111111111|
|273132200|2.777777777777778E-4|
|273327400|   7.611111111111111|
|309473000| 0.04083333333333333|
|563450000|  24.842777777777776|
|249879000| 0.08222222222222222|
|215397000|   3.575833333333333|
|311794000|  0.0961111111111111|
|311893000| 0.07666666666666666|
|636012382| 0.10027777777777777|
|219265000| 0.01888888888888889|
|257351000|                 0.1|
|257351000|  24.851111111111113|
|257427000| 0.09027777777777778|
|258962000|               53.82|
|265427000| 0.07166666666666667|
|244282000|2.777777777777778E-4|
+---------+--------------------+
only showing top 20 rows



In [9]:
port_congestion = waiting_time.withColumn(
    "rounded_lat", round(col("Latitude"), 2)
).withColumn(
    "rounded_long", round(col("Longitude"), 2)
).groupBy("rounded_lat", "rounded_long").agg(
    avg("waiting_duration").alias("avg_waiting_time"),
    count("*").alias("ship_count")
)


port_congestion.show()


+-----------+------------+--------------------+----------+
|rounded_lat|rounded_long|    avg_waiting_time|ship_count|
+-----------+------------+--------------------+----------+
|      59.31|        5.32| 0.04083333333333333|         1|
|      54.37|       10.99| 0.03861111111111111|         1|
|      53.92|       11.43|   7.611111111111111|         1|
|      59.33|       10.52| 0.07722222222222222|         6|
|      55.37|       13.15|  0.0661111111111111|         1|
|       55.7|       10.83| 0.10277777777777777|         1|
|      59.13|        9.62|2.777777777777778E-4|         1|
|       59.1|         9.6|   6.255486111111112|         4|
|      60.81|        5.02|  12.790833333333333|         2|
|      54.49|       13.59|   3.575833333333333|         1|
|      57.42|       10.78| 0.10027777777777777|         1|
|      54.15|       12.13| 0.07666666666666666|         1|
|      55.42|       13.83|  0.0961111111111111|         1|
|      60.82|        5.04|  24.851111111111113|         

In [10]:


port_congestion_rows = port_congestion.select("rounded_lat", "rounded_long", "ship_count", "avg_waiting_time").collect()

map_center = [sum(row["rounded_lat"] for row in port_congestion_rows) / len(port_congestion_rows),
              sum(row["rounded_long"] for row in port_congestion_rows) / len(port_congestion_rows)]

congestion_map = folium.Map(location=map_center, zoom_start=2)

for row in port_congestion_rows:
    folium.CircleMarker(
        location=(row["rounded_lat"], row["rounded_long"]),
        radius=row["ship_count"] * 2,  
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.6,
        popup=f"Avg Waiting Time: {row['avg_waiting_time']:.2f} hours\nShip Count: {row['ship_count']}"
    ).add_to(congestion_map)


congestion_map.save("port_congestion_map.html")
congestion_map
